## This notebook covers some of the use cases of RNN , LSTM and GRU cells

In [2]:
import tensorflow
from keras.layers.core import Dense, Activation
from keras.layers.recurrent import SimpleRNN
from keras.models import Sequential
from keras.utils.vis_utils import plot_model
import numpy as np

Using Theano backend.


### Read the text and create a vocab of characters and dictionary

In [7]:
fin = open("/Users/tkmacl9/Desktop/FastAIDLCourse/nbs/KerasDeepLearning/alice_in_wonderland.txt", "r")
lines = []
for line in fin:
    line.strip().lower().replace("\n","")
    line.encode("ascii", errors="ignore")
    if len(line)==0:
        continue
    lines.append(line)
fin.close()
# lines
text = " ".join(lines)
text.replace("\n","")
print(len(text))
# print(type(text))

167551


In [3]:
# Create the dictionary for char to index
chars = set([c for c in text])
nb_chars = len(chars)
char2index = dict((c,i) for i,c in enumerate(chars))
index2char = dict((i,c) for i,c in enumerate(chars))

In [26]:
print("Number of Characters - ", nb_chars)
print(len(char2index))
print(len(index2char))
print("")
print(char2index)

Number of Characters -  86
86
86

{'Z': 0, 'e': 1, 'x': 2, 'd': 44, 'L': 45, '[': 46, 'q': 47, 'P': 48, 'Y': 4, 'G': 49, '.': 5, 'm': 6, 'i': 50, '5': 8, 'T': 51, 'U': 52, 'R': 9, ':': 53, '“': 10, ']': 85, '0': 54, 'r': 11, '#': 55, 'O': 73, 'l': 13, 'y': 56, 'A': 14, '?': 15, 'w': 57, '9': 58, 's': 16, 'J': 18, '7': 59, ';': 60, 'z': 61, 'N': 62, ')': 63, 'a': 64, '6': 21, '_': 22, 'j': 3, 'F': 65, 'H': 23, '’': 19, 'B': 24, '(': 66, '1': 67, 'f': 68, '”': 25, '*': 12, 'M': 26, '\n': 70, 'Q': 71, 'k': 72, 't': 27, 'W': 28, '!': 74, 'I': 29, 'c': 75, '4': 30, 'h': 76, '-': 31, '@': 32, 'v': 33, '8': 77, '%': 7, 'K': 34, '/': 35, 'E': 78, 'b': 20, 'n': 36, 'V': 37, 'S': 69, 'D': 38, '$': 39, 'p': 40, 'o': 17, 'u': 79, '‘': 80, '3': 81, 'X': 82, '2': 83, ',': 84, 'g': 41, 'C': 42, ' ': 43}


### Convert the text in the form of sequences for the training sets and labels

In [33]:
# Define some of the Parameters
SEQLENGTH = 10
BATCH_SIZE = 128
HIDDEN_NEURONS = 128
STEP = 1
NUM_ITERATIONS = 25
NUM_EPOCHS_PER_ITERATION = 1
NUM_PREDS_PER_EPOCH = 100

input_chars = []
label_chars = []

for i in range(0, len(text)-SEQLENGTH, STEP):
    input_chars.append(text[i: i + SEQLENGTH])
    label_chars.append(text[i + SEQLENGTH])

# to test the Input characters and the Label Character data look and feel
print(len(input_chars))
print(len(label_chars))
print("")
for ind in range(0,20):
    print(input_chars[ind] + " - "+ label_chars[ind])

167541
167541

Project Gu - t
roject Gut - e
oject Gute - n
ject Guten - b
ect Gutenb - e
ct Gutenbe - r
t Gutenber - g
 Gutenberg - ’
Gutenberg’ - s
utenberg’s -  
tenberg’s  - A
enberg’s A - l
nberg’s Al - i
berg’s Ali - c
erg’s Alic - e
rg’s Alice - ’
g’s Alice’ - s
’s Alice’s -  
s Alice’s  - A
 Alice’s A - d


### Convert the input_chars and the label_chars to vector formats to feed into the RNN model

In [35]:
X = np.zeros((len(input_chars), SEQLENGTH, nb_chars), dtype=np.bool)
Y = np.zeros(( len(label_chars), nb_chars), dtype=np.bool)

print(X.shape)
print(Y.shape)

# now create the X and the Y vectors which will be used as our datasets to traing the RNN model
for i, input_char in enumerate(input_chars):
    for j, ch in enumerate(input_char):
#         print(j," ",ch)
        X[i, j, char2index[ch]] = 1
    Y[i, char2index[label_chars[i]]] = 1

# print one of the values for X and Y vectors
print(X[0])
print(" ")
print(Y[0])

(167541, 10, 86)
(167541, 86)
[[False False False False False False False False False False False False
  False False False False False False False False False False False False
  False False False False False False False False False False False False
  False False False False False False False False False False False False
   True False False False False False False False False False False False
  False False False False False False False False False False False False
  False False False False False False False False False False False False
  False False]
 [False False False False False False False False False False False  True
  False False False False False False False False False False False False
  False False False False False False False False False False False False
  False False False False False False False False False False False False
  False False False False False False False False False False False False
  False False False False False False False False False False False

### Define the model with the required parameters

In [18]:
model = Sequential()
model.add(SimpleRNN(RNN_CELLS, return_sequences = False,input_shape=(SEQLENGTH, nb_chars), unroll=True))
model.add(Dense(nb_chars))
model.add(Activation("softmax"))
model.summary()

# have the model compile
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_1 (SimpleRNN)     (None, 128)               27520     
_________________________________________________________________
dense_1 (Dense)              (None, 86)                11094     
_________________________________________________________________
activation_1 (Activation)    (None, 86)                0         
Total params: 38,614
Trainable params: 38,614
Non-trainable params: 0
_________________________________________________________________


### Train the model and test it , in each epoch unless the output looks great

In [40]:
# now train the model for the required number of iterations and repeat the process
# by testing and getting the next character and appending it to the next input

for iteration in range(NUM_ITERATIONS):
    print("=" * 50)
    print("Iteration - ", iteration+1)
    model.fit(X, Y, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS_PER_ITERATION)
    
    # this section is to test the output at every iteration, and see how it starts making sense
    test_idx = np.random.randint(len(input_chars))
    test_sample = input_chars[test_idx]
    print("The test sample is - ", test_sample)
    print(test_sample, end="")
    for i in range(NUM_PREDS_PER_EPOCH):
        X_test = np.zeros((1, SEQLENGTH, nb_chars), dtype=np.bool)
        for i, ch in enumerate(test_sample):
            X_test[0, i, char2index[ch]] = 1
        pred = model.predict(X_test, verbose=0)[0]
        y_pred = index2char[np.argmax(pred)]
        print(y_pred, end="")
        test_sample = test_sample[1:] + y_pred
    print()
        
    

Iteration -  1
Epoch 1/1
167541/167541 [==============================] - 14s - loss: 1.4396    
The test sample is -   to hear h
 to hear his side and she was a little shall her side and she was a little shall her side and she was a little
Iteration -  2
Epoch 1/1
167541/167541 [==============================] - 14s - loss: 1.4334    
The test sample is -  ur pocket?
ur pocket?’ said the Caterpillar more the thing the thing the thing the thing the thing the thing the thing th
Iteration -  3
Epoch 1/1
167541/167541 [==============================] - 15s - loss: 1.4277    
The test sample is -  uestion wa
uestion was to herself to the sartly of the starked and the poor little sight of the starked and the poor litt
Iteration -  4
Epoch 1/1
167541/167541 [==============================] - 15s - loss: 1.4233    
The test sample is -   she gave 
 she gave of the same as in the same as in the same as in the same as in the same as in the same as in the sam
Iteration -  5
Epoch 1/1
167541/1675